<a href="https://colab.research.google.com/github/Rayson-LD/flower-python/blob/main/Flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import json
import shutil
from google.colab import drive

In [ ]:
drive.flush_and_unmount()
shutil.rmtree('dataset')

In [ ]:

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!unzip drive/MyDrive/flowers.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/gazania/image_04491.jpg  
  inflating: dataset/train/gazania/image_04492.jpg  
  inflating: dataset/train/gazania/image_04493.jpg  
  inflating: dataset/train/gazania/image_04494.jpg  
  inflating: dataset/train/gazania/image_04495.jpg  
  inflating: dataset/train/gazania/image_04496.jpg  
  inflating: dataset/train/gazania/image_04497.jpg  
  inflating: dataset/train/gazania/image_04498.jpg  
  inflating: dataset/train/gazania/image_04499.jpg  
  inflating: dataset/train/gazania/image_04500.jpg  
  inflating: dataset/train/gazania/image_04501.jpg  
  inflating: dataset/train/gazania/image_04503.jpg  
  inflating: dataset/train/gazania/image_04504.jpg  
  inflating: dataset/train/gazania/image_04505.jpg  
  inflating: dataset/train/gazania/image_04506.jpg  
  inflating: dataset/train/gazania/image_04507.jpg  
  inflating: dataset/train/gazania/image_04508.jpg  
  inflating: dataset/train/gazania/image_04509.j

In [ ]:
image_size = 225
batch_size = 90
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.3
)
train_generator = datagen.flow_from_directory(
    '/content/dataset/train',
    target_size = (image_size,image_size),
    batch_size = batch_size,
    subset = 'training'
)
val_generator = datagen.flow_from_directory(
    '/content/dataset/valid',
    target_size = (image_size,image_size),
    batch_size = batch_size,
    subset = 'validation'
)

Found 4636 images belonging to 102 classes.
Found 198 images belonging to 102 classes.


In [ ]:

label = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt','w') as f :
  f.write(label)
  print(label)

alpine sea holly
anthurium
artichoke
azalea
ball moss
balloon flower
barbeton daisy
bearded iris
bee balm
bird of paradise
bishop of llandaff
black-eyed susan
blackberry lily
blanket flower
bolero deep blue
bougainvillea
bromelia
buttercup
californian poppy
camellia
canna lily
canterbury bells
cape flower
carnation
cautleya spicata
clematis
colt's foot
columbine
common dandelion
corn poppy
cyclamen
daffodil
desert-rose
english marigold
fire lily
foxglove
frangipani
fritillary
garden phlox
gaura
gazania
geranium
giant white arum lily
globe thistle
globe-flower
grape hyacinth
great masterwort
hard-leaved pocket orchid
hibiscus
hippeastrum
japanese anemone
king protea
lenten rose
lotus
love in the mist
magnolia
mallow
marigold
mexican aster
mexican petunia
monkshood
moon orchid
morning glory
orange dahlia
osteospermum
oxeye daisy
passion flower
pelargonium
peruvian lily
petunia
pincushion flower
pink primrose
pink-yellow dahlia
poinsettia
primula
prince of wales feathers
purple coneflower

In [ ]:
image_shape = (image_size,image_size,3)
base_model = tf.keras.applications.MobileNetV2(input_shape = image_shape, include_top = False,weights ='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential(
    [
     base_model,
     tf.keras.layers.Conv2D(32,3,activation = 'relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.GlobalAveragePooling2D(),
     tf.keras.layers.Dense(102,activation = 'softmax'),
    ]
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs = 20

In [ ]:
history = model.fit(train_generator,
                    epochs = epochs,
                    validation_data = val_generator)

Epoch 1/20
52/52 [==============================] - 185s 4s/step - loss: 3.7921 - accuracy: 0.1840 - val_loss: 5.7596 - val_accuracy: 0.0152
Epoch 2/20
52/52 [==============================] - 184s 4s/step - loss: 2.4935 - accuracy: 0.4247 - val_loss: 7.4288 - val_accuracy: 0.0101
Epoch 3/20
52/52 [==============================] - 184s 4s/step - loss: 1.5751 - accuracy: 0.6195 - val_loss: 9.0483 - val_accuracy: 0.0000e+00
Epoch 4/20
52/52 [==============================] - 184s 4s/step - loss: 1.0652 - accuracy: 0.7373 - val_loss: 10.3335 - val_accuracy: 0.0000e+00
Epoch 5/20
52/52 [==============================] - 185s 4s/step - loss: 0.7332 - accuracy: 0.8145 - val_loss: 11.6309 - val_accuracy: 0.0000e+00
Epoch 6/20
52/52 [==============================] - 183s 4s/step - loss: 0.5564 - accuracy: 0.8630 - val_loss: 12.3733 - val_accuracy: 0.0000e+00
Epoch 7/20
52/52 [==============================] - 183s 4s/step - loss: 0.4510 - accuracy: 0.8852 - val_loss: 13.1335 - val_accuracy: 

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f :
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmp94rwzg2b/assets


INFO:tensorflow:Assets written to: /tmp/tmp94rwzg2b/assets


In [ ]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>